# Völuspá analysis

The aim of this notebook is to see how the use of NLP techniques may help philological studies. My first analysis is on Völuspá, the first poem of the Elder Edda, telling the destiny of the worlds.

In [2]:
# Set your own user path
USER_PATH = "/home/pi"


In [4]:
# Download Old Norse Corpora to cltk_data directory
from cltk.corpus.utils.importer import CorpusImporter
onc = CorpusImporter("old_norse")
onc.import_corpus("old_norse_texts_heimskringla")


In [ ]:
# $ ipython profile create
# $ ipython locate
# $ nano profile_default/ipython_config.py
# # Add it a the end of the file (without '#'):
# c.InteractiveShellApp.exec_lines = [
#    'import sys; sys.path.append("~/cltk_data/old_norse")'
#]
# # And... It's done!


In [14]:
# old_norse_text_perseus
# __import__('/home/pi/cltk_data/old_norse/text/old_norse_texts_heimskringla/text_manager')
import os
module_path = os.path.join(USER_PATH, "cltk_data/old_norse/text/old_norse_texts_heimskringla/")
here = os.getcwd()
os.chdir(module_path)
corpus_path = os.path.join(module_path, "Völuspá")
import text_manager
loader = text_manager.TextLoader(os.path.join("Sæmundar-Edda", "Völuspá"), "txt")
text = loader.load()
os.chdir(here)
print(text[:200])


1.
Hljóðs bið ek allar
helgar kindir,
meiri ok minni
mögu Heimdallar;
viltu at ek, Valföðr,
vel fyr telja
forn spjöll fira,
þau er fremst of man.
 
2.
Ek man jötna
ár of borna,
þá er forðum mik
fædda 


#### split text in paragraphs ans verses


In [30]:
import re
indices = [(m.start(0), m.end(0)) for m in re.finditer(r"[0-9]{1,2}\.", text)]  #re.findall(r"[0-9]{1,2}\.", text)
paragraphs = [text[indices[i][1]:indices[i+1][0]].split("\n") for i in range(len(indices)-1)]
paragraphs = [ [ verse for verse in paragraph if verse != "" and verse != "\xa0"] for paragraph in paragraphs ]
paragraphs[:3]

[['Hljóðs bið ek allar',
  'helgar kindir,',
  'meiri ok minni',
  'mögu Heimdallar;',
  'viltu at ek, Valföðr,',
  'vel fyr telja',
  'forn spjöll fira,',
  'þau er fremst of man.'],
 ['Ek man jötna',
  'ár of borna,',
  'þá er forðum mik',
  'fædda höfðu;',
  'níu man ek heima,',
  'níu íviðjur,',
  'mjötvið mæran',
  'fyr mold neðan.'],
 ['Ár var alda,',
  'þat er ekki var,',
  'var-a sandr né sær',
  'né svalar unnir;',
  'jörð fannsk æva',
  'né upphiminn,',
  'gap var ginnunga',
  'en gras hvergi.']]

#### Use the phonology module to transcribe phonetically the text

#### show alliterations in the text

#### show vocabulary

#### show syntax